# Measuring Cache Sizes and Latency

Here's the CFiddle version of classic experiment to measure the size and latency of a system's caches.

In [ ]:
%xmode Minimal
from cfiddle import *

## Build A Randomize Circular Linked List

Here's the source:

In [ ]:
exe = build("misses/misses.cpp", arg_map(OPTIMIZE="-O3"))
exe[0].source()

## Check The Assembly

As you can see, this loop in `miss()` can't get much shorter, and the `movq` is not dependent on anything but itself.  Since the `movq` will be the longest-latency instruction and the processor is out-of-order, the latency should be solely determined by the `movq` latency.

Having easy access to the assembly makes it easy to verify that our latency measurements will mean what we want them to.

In [ ]:
exe[0].cfg("miss")

## Measure Performance for Varying Working Set Sizes

In [ ]:
results = run(exe, "go", arg_map(working_set=exp_range(1024, 1024*1024*4, 2**0.5), 
                                     iterations=100000000))
df = results.as_df()

## Graph it!

In [ ]:
df['ns/access'] = df["ET"]/df["iterations"]*1e9
df.plot.line(y="ns/access", x="working_set", ylabel="ns/access", logx=True, logy=False)

## Measure In Cycles

If you performance counters work on your system, you can measure latency in cycles as well:

In [ ]:
results = run(exe, "go", arg_map(working_set=exp_range(1024, 1024*1024*4, 2**0.5), 
                                     iterations=100000000),
                         perf_counters=["CYCLES"])
df = results.as_df()
df['cycles/access'] = df["CYCLES"]/df["iterations"]
df.plot.line(y="cycles/access", x="working_set", ylabel="cycles/access", logx=True, logy=False)